In [3]:
import panel as pn
import numpy as np
import pandas as pd
from pathlib import Path

# from matplotlib.figure import Figure
# from matplotlib.path import Path as mplPath
# from matplotlib import cm
import plotly.graph_objects as go

from file_paths import *


pn.extension("plotly")

data_dir = Path(
    "/Users/michaeldini/Dev/finance-dash-app/fin_dash_app/data/raw-ticker-max-daily"
)

ModuleNotFoundError: No module named 'file_paths'

In [2]:
def load_all_tickers_to_df(path: Path):
    files = path.glob("*_data.csv")
    df = pd.concat(
        [pd.read_csv(file) for file in files],
        ignore_index=True,
    )
    df.Date = pd.to_datetime(df.Date)
    df = df.set_index(["ticker", "Date"]).sort_index()
    return df

In [3]:
# sp500 = load_all_tickers_to_df(data_dir)
# get the past 90 days of data

# start = pd.Timestamp.now() - pd.Timedelta(days=90)
# sp500_2 = sp500.loc[(slice(None), slice(start, None)), :]

In [4]:
# sp500_3 = (
#     sp500_2.assign(
#         next_day_open=sp500_2.Open.groupby("ticker").shift(periods=-1),
#         change=lambda x: ((x.next_day_open - x.Close) / x.Close),
#         abs_change=lambda x: x.change.abs(),
#     )
#     .round(3)
#     .dropna()
# )
# print(
#     "If this was done as expected, there should be one dropped row for each ticker",
#     f"The number of dropped rows is: {len(sp500_2) - len(sp500_3)}",
# )

In [5]:
def find_fill_date(row, df):
    df = df.loc[row.name + pd.Timedelta(days=1) :]
    if row.change > 0:
        end = df[df.Low <= row.Close].head(1)
    else:
        end = df[df.High >= row.Close].head(1)

    if len(end) == 0:
        return pd.NaT
    else:
        return end.index[0]


# t = []
# for ticker in sp500_3.index.get_level_values("ticker").unique():
#     data = sp500_3.loc[ticker].copy()
#     data["ticker"] = ticker
#     data = data.assign(fill_date=data.apply(find_fill_date, args=[data], axis=1))
#     t.append(data)

# new_df = pd.concat(t)
# new_df.reset_index().to_csv('3monthanalysis.csv', index=False)

In [6]:
sp500 = (
    pd.read_csv(
        "3monthanalysis.csv", parse_dates=["Date"], dtype={"ticker": "category"}
    )
    .set_index(["ticker"])
    .sort_index()
)

In [7]:
sp500.head()

,Date,Open,High,Low,Close,Adj Close,Volume,next_day_open,change,abs_change,fill_date
ticker,,,,,,,,,,,
A,2023-04-28,133.45,136.93,133.45,135.43,135.172,2032800,136.02,0.004,0.004,2023-05-02
A,2023-05-01,136.02,137.14,135.76,136.08,135.821,1012900,135.02,-0.008,0.008,2023-05-02
A,2023-05-02,135.02,137.16,134.26,134.52,134.264,1629600,135.89,0.010,0.010,2023-05-04
A,2023-05-03,135.89,136.54,134.63,134.77,134.513,1529100,135.38,0.005,0.005,2023-05-04
A,2023-05-04,135.38,136.04,133.76,133.88,133.625,1103700,135.30,0.011,0.011,2023-05-05


In [8]:
# unfilled = sp500[sp500.fill_date.isna()]
# unfilled.loc["NVDA"]

In [9]:
# from itertools import cycle


def create_plot(ticker="NVDA", threshold=1):
    threshold = (threshold + 0.01) / 100
    data = sp500.loc[ticker].set_index("Date")

    # add extra dates to avoid keyerrors
    # not working. shifts the df and removes the first ten days
    # data.index = data.index + pd.Timedelta(days=10)
    start, stop = data.index.min(), (data.index.max() + pd.offsets.BusinessDay(1))
    date_range = pd.date_range(start, stop)
    data = data.reindex(date_range)
    # get the days that went unfilled and filter threshold paramter
    unfilled = data[data.fill_date.isna()].query("abs_change >= @threshold")

    # create three lists to create the gap vrect and line
    xs = []
    ys = []
    vrect_xs = []
    for date, e in unfilled[["Close", "Open"]].iterrows():
        tom = date + pd.Timedelta(days=1)
        xs.extend([date, tom, None])
        ys.extend([e.Close, data.loc[tom].Open, None])
        vrect_xs.append((date, tom))

    # plotly playout
    layout = go.Layout(
        title="",
        autosize=False,
        width=1200,
        height=600,
        margin=dict(t=50, b=50, r=50, l=50),
        xaxis_rangeslider_visible=False,
    )

    candlestick = go.Candlestick(
        x=data.index,
        open=data.Open,
        high=data.High,
        low=data.Low,
        close=data.Close,
        # opacity=1,
    )

    gap_lines = go.Scatter(
        x=xs,
        y=ys,
        mode="lines",
        line={"width": 3},
        marker_color="rgb(0,0,0)",
    )

    # this is how plotly wants a figure
    fig = go.Figure(data=[candlestick, gap_lines], layout=layout)

    # add the vrects to the figure
    for (
        today,
        tom,
    ) in vrect_xs:
        fig.add_vrect(
            x0=today,
            x1=tom,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        ),

    # create the panel and return it
    plotly_pane = pn.pane.Plotly(fig)
    return plotly_pane


# ticker choice widget
tickers = sp500.index.get_level_values("ticker").unique().to_list()
ticker_widget = pn.widgets.Select(name="ticker", value="NVDA", options=tickers)

# threshold widget
threshold_widget = pn.widgets.IntSlider(name="threshold", value=3, start=0, end=10)

bound_plot = pn.bind(create_plot, ticker=ticker_widget, threshold=threshold_widget)
app = pn.Column(ticker_widget, threshold_widget, bound_plot)

app

BokehModel(combine_events=True, render_bundle={'docs_json': {'1b40c91a-ce4b-4b0b-bf05-db38ec27974b': {'version…